<div align="center"; span style="color:#336699"><b><h2> Algorithm Workflow </h2></b></div>
<hr style="border:2px solid #0077b9;">
<br/>
<div style="text-align: center;font-size: 90%;">
    Helvécio B. Leal Neto, <sup><a href="https://orcid.org/0000-0002-7526-2094"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    Alan J. P. Calheiros<sup><a href="https://orcid.org/0000-0002-7526-2094"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:helvecio.neto@inpe.br">helvecio.neto@inpe.br</a>, <a href="mailto:alan.calheiros@inpe.br">alan.calheiros@inpe.br</a>
    <br/><br/>
    Last Update: Jun 16, 2024
</div>
<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter notebook demonstrates the processing flow of the pyForTraCC algorithm. There you can check the example of tracking using radar data.
</div>    
<br/>
<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook was based on the <a href="https://github.com/fortracc-project/pyfortracc">"pyfortracc"</a> example from the pyFortracc:</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
      <p> Leal Neto, H.B.; Calheiros, A.J.P.;  pyForTraCC Algorithm. São José dos Campos, INPE, 2024. <a href="https://github.com/fortracc-project/pyfortracc" target="_blank"> Online </a>. </p>
    </div>
</div>

### Schedule

 [1. Radar Data](#data)<br>
 [2. Setting the Environment](#install)<br>
 [3. Read Function](#read)<br>
 [4. Tracking Parameters](#parameters)<br>
 [5. Algorithm Workflow](#parameters)<br>
 [- Features Extraction](#features)<br>
 [- Spatial Operations](#spatial)<br>
 [- Cluster Link](#link)<br>
 [6. Explore Output](#output)

<a id='data'></a>
#### 1. Radar Data

The data in this example corresponds to a small sample of scans from the S-Band Radar located in the city of Manaus-AM Brazil.<br>
 Data processed and published by Schumacher, Courtney and Funk, Aaron (2018) were separated, <br>
 and are available in full on the ARM platform https://www.arm.gov/research/campaigns/amf2014goamazon.<br>
 This data is part of the GoAmazon2014/5 project and is named "Three-dimensional Gridded S-band Reflectivity and Radial Velocity<br>
 from the SIPAM Manaus S-band Radar dataset".<br>
https://doi.org/10.5439/1459573


In [ ]:
# Download the input files
!pip install --upgrade --no-cache-dir gdown &> /dev/null
!gdown 'https://drive.google.com/uc?id=1UVVsLCNnsmk7_wOzVrv4H7WHW0sz8spg'
!unzip -qq -o input.zip
!rm -rf input.zip

<a id='install'></a>
#### 2. Setting the environment

Install package to environment and import the package.

In [ ]:
# Run this cell to install the latest version of pyfortracc from the main branch
# !pip install --upgrade git+https://github.com/fortracc-project/pyfortracc.git@main#egg=pyfortracc
# Or import the local version of pyfortracc
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../../')

In [ ]:
# Import the pyfortracc package
import pyfortracc

<a id='read'></a>
#### 3. Read Function

The downloaded data is compressed with the .gz extension, however, it is of the netCDF4 type. The variable that represents reflectivity is DBZc. This data also has multiple elevations, and we arbitrarily chose elevation 5, which corresponds to the volumetric scan level at 2.5 km height. We extract the data and apply a NaN value to the data mask -9999.

In [ ]:
# Define the Read function
import gzip
import netCDF4
import numpy as np

def read_function(path):
    variable = "DBZc"
    z_level = 5 # Elevation level 2.5 km
    with gzip.open(path) as gz:
        with netCDF4.Dataset("dummy", mode="r", memory=gz.read()) as nc:
            data = nc.variables[variable][:].data[0,z_level, :, :][::-1, :]
            data[data == -9999] = np.nan
    gz.close()
    return data

In [ ]:
# Visualize the data
pyfortracc.plot_animation(path_files='input/*.gz', 
                          num_frames=10, figsize=(4, 4), cbar_min=-10, cbar_max=60,
                          read_function=read_function,  cbar_title='dBZ', cmap='viridis')

<a id='parameters'></a>
#### 4. Tracking Parameters

For this example, we will track reflectivity clusters at multiple thresholds and sizes <br>Arbitrarily selecting thresholds of 20, 30 and 35 dBZ with clusters of 5,4 and 3 pixels <br>of minimum size. The segmentation operator will be >=, that is, the clusters will be <br>segmented in order of greatest equal for each threshold and the delta of the observations <br>is 12 minutes. The clustering algorithm in this example will be DBSCAN and the overlap <br>rate between clusters of consecutive times will be 20%.

In [ ]:
# Example Name list dictionary of mandatory parameters
name_list = {}
name_list['input_path'] = 'input/' # path to the input data
name_list['output_path'] = 'output/' # path to the output data
name_list['timestamp_pattern'] = 'sbmn_cappi_%Y%m%d_%H%M.nc.gz' # timestamp file pattern
name_list['thresholds'] = [20,30,35] # in dbz
name_list['min_cluster_size'] = [3,3,3] # in number of points per cluster
name_list['operator'] = '>=' # '>= - <=' or '=='
name_list['delta_time'] = 12 # in minutes
name_list['cluster_method'] = 'dbscan' # DBSCAN Clustering method
name_list['min_overlap'] = 20 # Minimum overlap between clusters in percentage

<a id='workflow'></a>
#### 5. Algorithm Workflow

In this example we will use the modules separately, that is, each internal module of the pyForTraCC algorithm will be called individually.<br>
The track workflow is divided into four modules:
<ol>
  <li><b>Feature detection</b>: Focuses on identifying distinct characteristics for precise tracking.
  </li>
  <li><b>Spatial Operations</b>: Involves spatial operations (intersection, union, difference, etc) between the features of consecutive time steps (t and t-1).
  <li><b>Trajectory Linking</b>: Passing one by one the time steps, the algorithm link the features of consecutive time steps based on the association created in the previous step. The algorithm create a trajectory for each feature.
  <li><b>Concatenate</b>:
  </li>
</ol>

##### - Features Extraction
Image processing strategies were combined with clustering and rasterization algorithms to achieve the results obtained by the extract_features function. The following sequence is present in the implementation of the algorithm:

1. Read the file using the read_funcion.
2. Image segmentation according to each threshold.
3. Labeling of clusters. Two clustering options are implemented (DBSCAN and ndimage). 
5. Vectorization of the clusters using rasterio.features.shapes to acquire the boundary polygon and the centroid of the clusters.

<span> <img src="./img/features.png" style="height:320px;" align="left"></span>

In [ ]:
# (Note: If you are running this notebook using MacOS, you may need to set parallel=False)
pyfortracc.features_extraction(name_list, read_function, parallel=True)

The output

In [ ]:
import pandas as pd
import glob

features = sorted(glob.glob(name_list['output_path'] + '/track/processing/features/*.parquet'))
features = pd.concat(pd.read_parquet(f) for f in features)
display(features.head())

##### - Spatial operations 

Spatial junctions, fundamental in Geographic Information Systems (GIS) and spatial databases. And the Geopandas implementation simplifies the process by combining GeoDataframes stored in .parquet files based on their spatial relationships via the sjoin() method. This method performs various types of spatial joins such as overlays, within, contains.

To demonstrate the basic functioning of the spatial operations mode, we will use as a base two consecutive times listed here as the variables cur_frame and prev_frame, which store information about the characteristics of the systems for each time.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.wkt import loads

In [ ]:
# Read the features parquet files
cur_frame = pd.read_parquet('./output/track/processing/features/20140816_1012.parquet')
prev_frame = pd.read_parquet('./output/track/processing/features/20140816_1000.parquet')
cur_frame['geometry'] = cur_frame['geometry'].apply(loads)
prev_frame['geometry'] = prev_frame['geometry'].apply(loads)
# Convert to geo dataframes where column is geometry
cur_frame = gpd.GeoDataFrame(cur_frame)
prev_frame = gpd.GeoDataFrame(prev_frame)

In [ ]:
# Visualize the features and see have a visual overlap between the geometries
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
prev_frame.boundary.plot(ax=ax, color='blue', alpha=0.5)
cur_frame.plot(ax=ax, color='red', alpha=0.5)

In [ ]:
# Display the first 5 rows of the current frame
display(cur_frame.head())

In [ ]:
# Display the first 5 rows of the previous frame
display(prev_frame.head())

To demonstrate one of the operations performed in the algorithm, below is the GeoPandas sjoin function that checks the overlaps between the two GeoDataframes. 
Note that the return of the function will be another GeoDataframe, however only the index and index_right columns are of interest to us, 
as in these columns we have the information we need to make the associations between the geometries of the consecutive time clusters.
In addition to the overlap operation, there are several others that can be seen at:
https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates

In [ ]:
# Perform the spatial join between the previous and current frame
overlaps = gpd.sjoin(cur_frame, prev_frame, how='inner', predicate='overlaps')[['index_right']].reset_index()
# Ex: index (cur_frame) 0 overlaps with index_right 0 (prev_franme)
overlaps

In addition, the spatial operations module also has additional vector extraction methods. These methods are covered in the work https://doi.org/10.3390/rs14215408

To activate the methods just add the flags to the name_list.

In [ ]:
# Add correction methods
name_list['spl_correction'] = True # It is used to perform the correction at Splitting events
name_list['mrg_correction'] = True # It is used to perform the correction at Merging events
name_list['inc_correction'] = True # It is used to perform the correction using Inner Core vectors
name_list['opt_correction'] = True # It is used to perform the correction using the Optical Flow method
name_list['validation'] = True # It is used to perform the validation of the correction methods

In [ ]:
# Run the spatial_operations function (Note: If you are running this notebook using MacOS, you may need to set parallel=False)
pyfortracc.spatial_operations(name_list, read_function, parallel=True)

In [ ]:
# Read the spatial parquet files
spatial = sorted(glob.glob(name_list['output_path'] + '/track/processing/features/*.parquet'))
spatial_df = pd.concat(pd.read_parquet(f) for f in spatial)
display(spatial_df.head())

##### Cluster Link

The cluster connection module makes the association between consecutive time tables by associating the cluster indices that were identified by the spatial operations module.

In [ ]:
# Run the cluster_linking function
pyfortracc.cluster_linking(name_list)

Bellow we show how cluster link works. Set two spatial parquets from consecutive timestamps. And show the association between them based on indexes.

In [ ]:
# Read current and previous frames
cur_frame = pd.read_parquet('output/track/processing/spatial/20140816_1212.parquet')
prv_frame = pd.read_parquet('output/track/processing/spatial/20140816_1200.parquet')

In [ ]:
# Current Frame
cur_frame.dropna(subset=['prev_idx']).head()

In [ ]:
# Previous frame
prv_frame.loc[cur_frame['prev_idx'].dropna().astype(int).values].head()

In [ ]:
# Show the linking results
linked = sorted(glob.glob(name_list['output_path'] + '/track/processing/linked/*.parquet'))
linking_df = pd.concat(pd.read_parquet(f) for f in linked)
linking_df.loc[linking_df['trajectory'] != 'LINESTRING EMPTY'].tail()

#### Concatenate 

All features in one single file per timestamp, and create the Tracking Table.


In [ ]:
# Concatenate the features and spatial dataframes
pyfortracc.concat(name_list, clean=True)

The tracking table is the generalized output entity of the algorithm, it is formed by the set of files (.parquet) that are located in the output directory of the same name ('output_path/trackingtable'). The information obtained in the tracking process is stored in a tabular format, and is organized according to the tracking time. Listed below are the names of the columns (output variables) and what they represent.

- Each row in the tracking specific data related to a cluster at its corresponding threshold level. 
- The information spans from unique identifiers and descriptive statistics to geometric properties and temporal features. 
- The Tracking Table structure provides a comprehensive view of grouped entities, facilitating analysis and understanding of patterns across different threshold levels.

Tracking table columns:

*   **timestamp** (datetime64[us]): Temporal information of cluster.
*   **uid** (float64): Unique idetifier of cluster.
*   **iuid** (float64): Internal Unique idetifier of cluster.
*   **threshold_level** (int64): Level of threshold.
*   **threshold** (float64): Specific threshold.
*   **lifetime** (timedelta64[ns]): Cluster lifespan.
*   **status** (object): Entity status (NEW, CONTINUOUS, SPLIT, MERGE, SPLIT/MERGE)
*   **u_, v_** (float64): Vector components.
*   **inside_clusters** (object): Number of inside clusters.
*   **size** (int64): Cluster size in pixels.
*   **min, mean, max, std, Q1, Q2, Q3** (float64): Descriptive statistics.
*   **delta_time** (timedelta64[us]): Temporal variation.
*   **file** (object): Associated file name.
*   **array_y, array_x** (object): Cluster array coordinates.
*   **vector_field** (object): Associated vector field.
*   **trajectory** (object): Cluster's trajectory.
*   **geometry** (object):  Boundary geometric representation of the cluster.

In [ ]:
tracking_files = sorted(glob.glob(name_list['output_path'] + '/track/trackingtable/*.parquet'))
tracking_table = pd.concat(pd.read_parquet(f) for f in tracking_files)
display(tracking_table.head())

In [ ]:
# If need to save the family table into separate files or unique file run the cell below

import pandas as pd
import glob
import pathlib

tracking_files = sorted(glob.glob(name_list['output_path'] + '/track/trackingtable/*.parquet'))
tracking_table = pd.concat(pd.read_parquet(f) for f in tracking_files)

family_group = tracking_table.groupby('uid')
family_table = pd.DataFrame()
pathlib.Path('output/track/trackingtable/family').mkdir(parents=True, exist_ok=True)
for _, group in family_group:
    family_table = pd.concat([family_table, group])
    # # If need save in separate files uncomment the line below
    # uid_ = group['uid'].iloc[0]
    # group.to_csv(f'output/track/trackingtable/family/{uid_}.csv')
# If need save into unique file uncomment the line below
family_table.to_csv('output/track/trackingtable/family.csv')

#### Explore the results in tracking table

In [ ]:
# Get two maxlifetime clusters from the track_table
maxlifetime = 2
max_lifetimes = tracking_table.groupby('uid').size().nlargest(maxlifetime).index.values
max_clusters = tracking_table[tracking_table['uid'].isin(max_lifetimes)]
max_clusters.loc[max_clusters['threshold_level'] == 0].head()

#### Visualize the track as animation

In [ ]:
# To add plot the tracking data
name_list['lon_min'] = -62.1475 # Min longitude of data in degrees
name_list['lon_max'] = -57.8461 # Max longitude of data in degrees
name_list['lat_min'] = -5.3048 # Min latitude of data in degrees
name_list['lat_max'] = -0.9912 # Max latitude of data in degrees
name_list['x_dim'] = 241 # Number of points in the x axis
name_list['y_dim'] = 241 # Number of points in the y axis

In [ ]:
# Plot the tracking data for periods of time
pyfortracc.plot_animation(read_function=read_function, name_list=name_list, 
                          figsize=(14,5), cbar_title='dBZ', vector=True, vector_scale=10,
                          threshold_list=[20], uid_list=max_lifetimes.tolist(),
                          info=True, info_col_name=True, info_cols=['uid', 'method', 'far'],
                          smooth_trajectory=True,
                          start_stamp = '2014-08-16 12:24:00', 
                          end_stamp='2014-08-16 20:48:00')

#### Convert the results to spatial data type.

In [ ]:
pyfortracc.spatial_conversions(name_list, boundary=True, trajectory=True, vector_field=True, cluster=True, vel_unit='m/s', driver='GeoJSON')